# 函数装饰和闭包

In [1]:
# 函数装饰器用于在源码中“标记”函数，以某种方式增强函数行为
# 本质就是母函数修饰子函数，母函数的输入是一个函数，返回另一个函数

In [ ]:
@decorate
def target():
    print ('running target()')

In [ ]:
#等同于
def target():
    print ('running target()')

target = decorate(target)

In [5]:
# 装饰器把函数替换成另一个函数
def deco(func):
    def inner():
        print ('running inner()')
    return inner

@deco
def target():
    print ('running target()')

In [8]:
# 函数装饰器在导入模块时立即执行，而被装饰的函数，只在明确调用时运行
registry = []

def register(func):
    print ('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print ('running f1')

@register
def f2():
    print ('running f2')

def f3():
    print ('running f3')

running register(<function f1 at 0x10f7ee0d0>)
running register(<function f2 at 0x10f7ee488>)


In [9]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [ ]:
# 用装饰器解决上一章，添加新策略的问题
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    pass

@promotion
def bulk_item(order):
    pass

@promotion
def large_order(order):
    pass

def best_promo(order):
    return max(promo(order) for promo in promos)

# 变量作用域

In [1]:
b = 6
def f1(a):
    print (a)
    print (b)
f1(3)

3
6


In [3]:
b = 6
def f2(a):
    print (a)
    print (b)
    b = 3 # 假定在函数定义体中赋值的变量是局部变量
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [4]:
b = 6
def f3(a):
    global b
    print (a)
    print (b)
    b = 9
f3(3)
print (b)

3
6
9


In [1]:
class Averager():
    def __init__(self):
        self.series =[]
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [2]:
def make_averager():
    # 闭包 start
    series = []
    def averager(new_value):
        series.append(new_value) # 此时series是自由变量(free variable)
        total = sum(series)
        return total / len(series)
    return averager
    # 闭包 end

In [3]:
avg = make_averager()

In [5]:
print ("本地变量:", avg.__code__.co_varnames)
print ("自由变量:", avg.__code__.co_freevars)

本地变量: ('new_value', 'total')
自由变量: ('series',)


In [6]:
avg(10)
avg(11)
avg(12)

11.0

In [7]:
# __closure__属性各个元素对应于avg.__code__.co_freevars中的一个名称。这些元素都是cell对象
avg.__closure__

(<cell at 0x10c800b58: list object at 0x10caabc08>,)

In [8]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [16]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total 
        count += 1
        total += new_value
        return total / count
    return averager

In [17]:
avg = make_averager()

In [19]:
avg.__closure__

(<cell at 0x10c7d6ee8: int object at 0x10996f580>,
 <cell at 0x10c7d6138: int object at 0x10996f580>)

In [20]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ','.join(repr(arg) for arg in args)
        print ('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [21]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [22]:
snooze(5)

[5.00512475s] snooze(5) -> None


# 标准库中的装饰器

In [23]:
# functools.lru_cache 做备忘
# 耗时的函数结果保存起来
# functools.lru_cache(maxsize = 128, typed = False) 为了达到最佳性能，maxsize 为2的幂
import functools
@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

In [24]:
fibonacci(6)

[0.00000069s] fibonacci(0) -> 0
[0.00000077s] fibonacci(1) -> 1
[0.00018995s] fibonacci(2) -> 1
[0.00000177s] fibonacci(3) -> 2
[0.00057809s] fibonacci(4) -> 3
[0.00000121s] fibonacci(5) -> 5
[0.00066464s] fibonacci(6) -> 8


8

In [1]:
# functools.singledispatch python 不支持重载，这个修饰器可以起到重载的作用
from functools import singledispatch
from collections import abc
import numbers
import html

In [ ]:
@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [3]:
registry = set()

def register(active = True):
    def decorate(func):
        print ('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active = False)
def f1():
    print ('running f1()')

@register()
def f2():
    print ('running f2()')

def f3():
    print ('running f3()')

running register(active=False)->decorate(<function f1 at 0x1067416a8>)
running register(active=True)->decorate(<function f2 at 0x1067417b8>)


In [ ]:
# 参数化
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt = DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ','.join(repr(arg) for arg in _args)
            result = repr(_result)
            print (fmt.format(**locals()))
            return _result
        return clocked
    return decorate
            